In [ ]:
import firedrake
mesh = firedrake.UnitSquareMesh(32, 32, diagonal='crossed')
V = firedrake.FunctionSpace(mesh, family='CG', degree=2)
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)

Generate the true solution and the synthetic observational data.

In [ ]:
from firedrake import Constant, cos, sin
import numpy as np
from numpy import pi as π
from numpy import random

seed = 1729
generator = random.default_rng(seed)

degree = 5
x = firedrake.SpatialCoordinate(mesh)

q_true = firedrake.Function(Q)
for k in range(degree):
    for l in range(int(np.sqrt(degree**2 - k**2))):
        Z = np.sqrt(1 + k**2 + l**2)
        ϕ = 2 * π * (k * x[0] + l * x[1])

        A_kl = generator.standard_normal() / Z
        B_kl = generator.standard_normal() / Z
        
        expr = Constant(A_kl) * cos(ϕ) + Constant(B_kl) * sin(ϕ)
        mode = firedrake.interpolate(expr, Q)
        
        q_true += mode

In [ ]:
from firedrake import exp, inner, grad, dx
u = firedrake.Function(V)
f = Constant(1.0)
J = (0.5 * exp(q_true) * inner(grad(u), grad(u)) - f * u) * dx
bc = firedrake.DirichletBC(V, 0, 'on_boundary')
F = firedrake.derivative(J, u)
firedrake.solve(F == 0, u, bc)
u_true = u.copy(deepcopy=True)

In [ ]:
num_points = 25
δs = np.linspace(-0.5, 2, num_points + 1)
X, Y = np.meshgrid(δs, δs)
xs = np.vstack((X.flatten(), Y.flatten())).T

θ = π / 12
R = np.array([
    [np.cos(θ), -np.sin(θ)],
    [np.sin(θ), np.cos(θ)]
])

xs = np.array([
    x for x in (xs - np.array([0.5, 0.5])) @ R
    if (0 <= x[0] <= 1) and (0 <= x[1] <= 1)
])

In [ ]:
xs.shape

In [ ]:
U = u_true.dat.data_ro[:]
u_range = U.max() - U.min()
signal_to_noise = 20
σ = firedrake.Constant(u_range / signal_to_noise)
ζ = generator.standard_normal(len(xs))
u_obs = np.array(u_true.at(xs)) + float(σ) * ζ

Our finite element mesh is way denser than the observational data.

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
firedrake.triplot(mesh, axes=axes, interior_kw={'linewidth': 0.25})
axes.scatter(xs[:, 0], xs[:, 1], color='tab:blue');

The bad thing that geophysicists will do is directly interpolate from the observational data to the finite element representation and treat this interpolated field as the "observations", when really quite a lot has been filled in.
It's even worse when you consider the fact that there are edge degrees of freedom too, since we used piecewise quadratic basis functions.
Let's try every interpolator in scipy and see what the differences look like.

In [ ]:
from scipy.interpolate import (
    LinearNDInterpolator,
    NearestNDInterpolator,
    CloughTocher2DInterpolator,
    Rbf,
)

element = Q.ufl_element()
Vc = firedrake.VectorFunctionSpace(mesh, element)
X = firedrake.interpolate(mesh.coordinates, Vc).dat.data_ro[:]

interpolators = {
    'nearest': NearestNDInterpolator(xs, u_obs),
    'linear': LinearNDInterpolator(xs, u_obs, fill_value=0.0),
    'clough-tocher': CloughTocher2DInterpolator(xs, u_obs, fill_value=0.0),
    'gaussian': Rbf(xs[:, 0], xs[:, 1], u_obs, function='gaussian'),
}

interpolated_data = {}
for method, interpolator in interpolators.items():
    u_interpolated = firedrake.Function(Q)
    u_interpolated.dat.data[:] = interpolator(X[:, 0], X[:, 1])
    interpolated_data[method] = u_interpolated

In [ ]:
fig, axes = plt.subplots(
    nrows=1, ncols=len(interpolated_data), sharex=True, sharey=True
)
for ax in axes:
    ax.set_aspect('equal')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
for ax, (method, u_o) in zip(axes, interpolated_data.items()):
    firedrake.tripcolor(u_o, axes=ax)
    ax.set_title(method)

In [ ]:
import firedrake_adjoint

results = {}
for method, u_o in interpolated_data.items():
    u = firedrake.Function(V)
    q = firedrake.Function(Q)
    J = (0.5 * exp(q) * inner(grad(u), grad(u)) - f * u) * dx
    bc = firedrake.DirichletBC(V, 0, 'on_boundary')
    F = firedrake.derivative(J, u)
    firedrake.solve(F == 0, u, bc)

    E = 0.5 * ((u - u_o) / σ)**2 * dx

    α = firedrake.Constant(0.05)
    R = 0.5 * α**2 * inner(grad(q), grad(q)) * dx
    J = firedrake.assemble(E + R)

    q̂ = firedrake_adjoint.Control(q)
    Ĵ = firedrake_adjoint.ReducedFunctional(J, q̂)

    q_min = firedrake_adjoint.minimize(
        Ĵ, method='Newton-CG', options={'disp': True}
    )
    
    results[method] = q_min

In [ ]:
fig, axes = plt.subplots(
    nrows=1, ncols=len(results), sharex=True, sharey=True
)
for ax in axes:
    ax.set_aspect('equal')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
for ax, (method, q) in zip(axes, results.items()):
    firedrake.tripcolor(q, axes=ax)
    ax.set_title(method)

To evaluate how much of an effect the different interpolation schemes have, we'll print out a matrix of relative distances between the results the true value of the parameters $q_{\text{true}}$.
In many cases the relative distance between two solutions obtained with different approximation methods is an appreciable fraction of the relative distance between one of those solutions and the true solution itself.
For example, the relative distance between the solutions $q_{\text{linear}}$ and $q_{\text{gaussian}}$ is about a 30\% of the relative distance between $q_{\text{linear}}$ and $q_{\text{true}}$.

In [ ]:
results['exact'] = q_true
results = list(results.items())

In [ ]:
print(f"{'': <15}| ", end="")
for method, _ in results:
    print(f"{method: <14}| ", end="")
print(f"{'':-<15}|" * 6)
    
for method1, q1 in results:
    print(f"{method1: <15}|", end="")
    for method2, q2 in results:
        error = firedrake.norm(q1 - q2) / np.sqrt(firedrake.norm(q1) * firedrake.norm(q2))
        print(f"{error: 14.3f} |", end="")
    print("")

Arbitrary algorithmic choices -- like whether you used linear interpolation on a triangulated irregular network or radial basis functions -- should not make such a huge difference in how we do parameter estimation.